### Import Packages and OpenAI API Key

In [ ]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv('api_key')

print("API Key:", api_key)

### Import the Preprocessed Donor List.
<p> 
The donor list has been preprocessed to format each of the responses in a first person voice. The original reponses were in int64, float64, and string format. 
<p>
As an example, in the original rough donor file, the feature number_gifts was stored as a integer value. In the preprocessed donor file, this integer value as been replaced with "I have given ______ times." So a value of 11, would be preprossed to returned "I have given 11 times." 
<p>
Missing values are returned as NA and omitted in the following script.

In [ ]:
# Load the CSV file containing donor data
df = pd.read_csv("donors.csv")

df.head()

### Select a sample of donors.
<p>
As an example, select a 20 person sample of donors randomly from the donor list.

In [ ]:
#random sample of 20 donors
sdf = df.sample(20)

sdf.head()

### Create the self description for each donor.
<p>
Using a conversational prompt format, this will orient the model to each donor's specific interests.

In [ ]:
#combine features not equal to NA into a single string for each donor with the exception of donor_id and name
sdf['description'] = sdf[sdf.columns[2:]].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)

sdf['description'].head()

### Define Writer
<p>
Using a conversational prompt technique, this will orient the model to who the writer is affecting the ouput of the model. 

In [ ]:
# Access the API key
writer = os.getenv('nep_writer')

print("Writer Bio:", writer)

### Define Client Org
<p>
Using a conversational prompt technique, this will orient the model to the organization soliciting the donation.

In [ ]:
client_org = os.getenv('nep_client')

print("Client Bio:", client_org)

### Generate messages via GPT4
<p>
This script iterates through each donor, using their specific demographic and giving information to create a solicitation letter in the voice of the writer defined earlier on behalf of the client defined earlier.

In [ ]:
client = OpenAI(api_key=api_key)

solicitation_df = pd.DataFrame(columns=['donor_id', 'solicitation'])

# Example usage
for index, row in sdf.iterrows():
    # combine columns not labeled name or donor_id into a single string of text
    demographic_info = row['description']

    #save name to variable
    name = row['name']

    # Creating a prompt using the demographic information
  
    prompt = f"Create a personalized solicitation message for {name},"
    prompt += " who describes themselves as: " + demographic_info +"."
    prompt += " The message should be tailored to their interests and background."
    prompt += " The message should be professional and respectful."
    prompt += " The message should be written in a friendly and engaging tone."
    prompt += " The message should be concise and to the point."
    prompt += " The message should be persuasive and compelling."
    prompt += " The writer should use the donors description of themselves in an inconscpicious way."
    prompt += " The message should feature a specific ask for support and be less than 1000 characters."

    #generate the message
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": writer},
            {"role": "user", "content": client_org},
            {"role": "user", "content": prompt}
        ],
       temperature=1,
       max_tokens=1000,
       top_p=1,
       frequency_penalty=0,
       presence_penalty=0
    )
    
    #save the message to new dataframe by joining with donor_id
    solicitation = response.choices[0].message.content 

    #add each response as a new row in the dataframe
    solicitation_df = pd.concat([solicitation_df, pd.DataFrame({'donor_id': row['donor_id'], 'solicitation': solicitation}, index=[0])], ignore_index=True)

### Output to CSV for mail merge.

In [ ]:
#save the dataframe to a csv file
solicitation_df.to_csv('solicitations.csv', index=False)

#save sample to a csv file
sdf.to_csv("sample.csv", index=False)